# Mastering Shiny — Handling Uploads, Downloads & Publishing Online

**Goal:**  
Learn how Shiny handles **file uploads**, **downloads**, **bookmarking**, and **publishing apps online**.  

---

## What you’ll learn

### 🗂️ File uploads
- How to use `fileInput()` and explore what `input$upload` returns.  
- Preview uploaded files and understand the metadata returned (name, type, size, datapath).  

### ✅ Validating uploads
- Check file type, size, and contents safely on the **server side**.  
- Read and process uploaded data (CSV or TSV) using `vroom`.  

### 💾 Downloads
- Create downloadable files with `downloadHandler()` and `downloadButton()`.  
- Let users save their data or results (e.g., CSVs or plots).  

### 📌 Bookmarking (saving app state)
- Enable users to **save or share a link** that restores the same app state (selected variables, inputs).  
- Use `bookmarkButton()` and `enableBookmarking = "url"` for shareable bookmarks.

### 🌐 Publishing Online
- Deploy your Shiny app to the web using **Shinyapps.io** 🔗

### 🧩 Practical exercise
A hands-on project that combines everything:
- Upload a CSV file.  
- Select a variable and display a histogram.  
- Download the plot as `.png` or `.pdf`.  
- Optionally, use bookmarking to save and restore the app’s state.


In [3]:
# Prerequisites: run this cell first (R)
# Install packages if you don't have them. Uncomment the install lines if needed.
# install.packages(c('shiny', 'vroom'))

# Quick session check. 
sessionInfo()
# It reports details about your current R environment — it’s a diagnostic tool that tells you exactly what version of R and packages you’re using.

R version 4.5.1 (2025-06-13)
Platform: x86_64-conda-linux-gnu
Running under: Ubuntu 22.04.4 LTS

Matrix products: default
BLAS/LAPACK: /opt/conda/lib/libmkl_rt.so.2;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

time zone: Europe/Berlin
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] digest_0.6.37     IRdisplay_1.1     base64enc_0.1-3   fastmap_1.2.0    
 [5] glue_1.8.0        htmltools_0.5.8.1 repr_1.1.7        lifecycle_1.0.4  
 [9] cli_3.6.5         vctrs_0.6.5       textshaping_1.0.3 pbdZMQ_0.3-14    
[13] systemfonts_1.3.1 compiler_4.5.1    tools_4.5.1       ragg_1.5.0       

## 1 — Understanding file uploads

**Concepts**
- `fileInput()` creates a widget that lets users pick files.
- On the server `input$<id>` (for example `input$upload`) is `NULL` until a file is uploaded.
- When present, `input$upload` is a small table-like structure (data frame) with columns:
  - `name` — the original filename,
  - `size` — size in bytes,
  - `type` — MIME type (file format) guessed by the browser, such as `text/csv` for a CSV file or `image/png` for a PNG image.,
  - `datapath` — a temporary path on the server where the uploaded file is stored.

**Run the app below** to upload files and inspect `input$upload`. It will render a table showing the structure and also print `names(input$upload)` so you can see columns.

**Activity:** Upload a couple of small files (images or CSVs) and inspect the table.

In [ ]:
# Minimal upload app (R)
library(shiny)

ui <- fluidPage(
  fileInput('upload', 'Upload files', multiple = TRUE, buttonLabel = 'Choose...'),
  tableOutput('files')
)

server <- function(input, output, session) {
  output$files <- renderTable({
    req(input$upload)
    input$upload
  })
}

shinyApp(ui, server)


## Enhanced minimal upload app (Optional)

The app below is a slightly more **structured version** of the previous minimal upload app.  

**Key differences from the first app:**
- Uses `titlePanel()` and `sidebarLayout()` for a clearer layout with sidebar and main panel.  
- Adds descriptive `helpText()` to guide the user.  
- Displays **column names** of `input$upload` using `verbatimTextOutput()` and `names(input$upload)`.  
- Uses `h4()` headings for better visual separation of sections.

This version helps to **visually understand the upload structure** and see column names alongside the uploaded data table.


In [ ]:

library(shiny)

ui <- fluidPage(
  titlePanel('Upload inspector — minimal'),
  sidebarLayout(
    sidebarPanel(
      fileInput('upload', 'Choose file(s)', multiple = TRUE),
      helpText('Upload any small file (CSV, image, txt).')
    ),
    mainPanel(
      h4('Uploaded files (full structure)'),
      tableOutput('files'),
      h4('Column names of input$upload'),
      verbatimTextOutput('cols')
    )
  )
)

server <- function(input, output, session) {
  output$files <- renderTable({
    req(input$upload)         # wait until a file is present
    input$upload              # this is a data.frame-like object
  })
  output$cols <- renderPrint({
    req(input$upload)
    names(input$upload)
  })
}

shinyApp(ui, server)



Listening on http://127.0.0.1:6777



### Explanation of new UI elements

- `titlePanel('Upload inspector — minimal')`: Adds a main title at the top of the page.  
- `sidebarLayout()` / `sidebarPanel()` / `mainPanel()`: Creates a **two-column layout** — inputs on the left, outputs on the right.  
- `helpText()`: Provides a small instruction text for the user below the input.  
- `h4()`: Adds headings above output sections for better visual separation.  
- `verbatimTextOutput('cols')`: Shows the **raw text output** of `names(input$upload)` in a fixed-width font, useful for inspection.  

This structured version makes it easier to **understand and navigate the uploaded data**, compared to the simple one-table-only version.


## 2 — Validating uploads and reading CSV/TSV

When users upload files in Shiny, it’s important to make sure the files are **safe and in the format your app expects**. Although the `accept` argument in `fileInput()` can suggest which types of files a user should pick, this check only happens in the browser and **cannot be fully trusted**. Users could rename files or bypass the browser’s restrictions entirely.

**Why server-side validation matters:**  
- Ensures uploaded files match the expected type and structure.  
- Prevents errors caused by malformed or unexpected files.  
- Allows you to check the file’s extension, size, and even content before processing.

To illustrate, the example below demonstrates a simple Shiny app that:  

- Accepts `.csv` or `.tsv` files (the `accept` argument still guides the user).  
- Validates the file extension on the server using `tools::file_ext()`.  
- Reads the file efficiently with `vroom`.  
- Displays the first `n` rows, letting the user control how much data they see.

This example shows how to combine **user-friendly UI hints** with **robust server-side checks**.



In [ ]:
# CSV/TSV uploader with validation (R)
library(shiny)
library(vroom)

ui <- fluidPage(
  fileInput('upload', NULL, accept = c('.csv', '.tsv')),
  numericInput('n', 'Rows', value = 5, min = 1),
  tableOutput('head')
)

server <- function(input, output, session) {
  data <- reactive({
    req(input$upload)
    ext <- tools::file_ext(input$upload$name)
    switch(ext,
      csv = vroom(input$upload$datapath, delim = ','),
      tsv = vroom(input$upload$datapath, delim = '\t'),
      validate('Invalid file; Please upload a .csv or .tsv file')
    )
  })

  output$head <- renderTable({
    head(data(), input$n)
  })
}

shinyApp(ui, server)

### Explanation of the CSV/TSV upload app

**UI section (`ui <- fluidPage(...)`):**  
- `fileInput('upload', NULL, accept = c('.csv', '.tsv'))` – creates a file upload widget. The browser will guide the user to select CSV or TSV files.  
- `numericInput('n', 'Rows', value = 5, min = 1)` – lets the user choose how many rows of the uploaded file to display.  
- `tableOutput('head')` – placeholder in the UI where the top `n` rows of the file will be shown.  

**Server section (`server <- function(...)`)**:  
- `data <- reactive({ ... })` – defines a reactive expression that reads and validates the uploaded file. It updates automatically when a new file is uploaded.  
- `req(input$upload)` – ensures that the code below runs **only after a file is uploaded**; prevents errors if nothing is uploaded yet.  
- `ext <- tools::file_ext(input$upload$name)` – extracts the file extension (e.g., "csv" or "tsv") from the uploaded file’s name.  
- `switch(ext, ...)` – chooses how to read the file based on its extension:  
  - `csv = vroom(input$upload$datapath, delim = ',')` – reads CSV files.  
  - `tsv = vroom(input$upload$datapath, delim = '\t')` – reads TSV files.  
  - `validate('Invalid file; Please upload a .csv or .tsv file')` – stops execution and shows an error if the file type is invalid.  

- `output$head <- renderTable({ head(data(), input$n) })` – displays the first `n` rows of the uploaded file in the table output.  




## 3 — Creating downloads with `downloadHandler()`

Shiny allows users to **download data from your app** using a simple pattern:

- **UI:** Add a `downloadButton()` or `downloadLink()` where the user can click to download a file.  
- **Server:** Define a `downloadHandler()` that tells Shiny:
  - What filename to give the downloaded file (`filename` function), and  
  - What content to write to that file (`content` function).

The example below does the following:

1. `selectInput('dataset', 'Pick a dataset', choices = ls('package:datasets'))` – lets the user choose a dataset from R’s built-in `datasets` package.  
2. `tableOutput('preview')` – shows the first few rows of the selected dataset.  
3. `downloadButton('download', 'Download .tsv')` – provides a button to download the dataset as a TSV file.  

**Server logic:**

- `data <- reactive({ ... })` – reactive expression that gets the selected dataset and ensures it is a data frame.  
- `output$preview <- renderTable(head(data()))` – shows the first few rows for preview.  
- `output$download <- downloadHandler(...)` – defines how the download works:
  - `filename = function() paste0(input$dataset, '.tsv')` – sets the file name to the dataset name + `.tsv`.
  - `content = function(file) vroom::vroom_write(data(), file)` – writes the dataset to a TSV file that the user can download.

This app lets users **pick a dataset, see a preview, and download it safely** in a familiar format.



In [ ]:
# Dataset preview + download as CSV (R)

ui <- fluidPage(
  selectInput('dataset', 'Pick a dataset', choices = ls('package:datasets')),
  tableOutput('preview'),
  downloadButton('download', 'Download .csv')
)

server <- function(input, output, session) {
  data <- reactive({
    out <- get(input$dataset, 'package:datasets')
    if(!is.data.frame(out)) validate('Selected object is not a data frame')
    out
  })

  # Show first few rows for preview
  output$preview <- renderTable(head(data()))

  # Download handler for CSV
  output$download <- downloadHandler(
    filename = function() paste0(input$dataset, '.csv'),
    content = function(file) {
      vroom::vroom_write(data(), file, delim = ",")  # comma delimiter for CSV
    }
  )
}

shinyApp(ui, server)



## 4 — Practical exercise: Upload → Select Variable → Histogram → Download

**Challenge:**  

In this exercise, you will build a Shiny app that allows the user to:

1. **Upload a CSV file** from their computer.  
2. **Select a numeric variable** from the uploaded dataset using a dropdown.  
3. **Draw a histogram** of the selected variable.  
4. **Download the histogram** as a file (PNG or PDF).  

---

### Your task

Before looking at the solution, try to implement this on your own. Your app should include:

- A **file input** to upload the CSV file.  
- A **dropdown** that lists only numeric variables from the uploaded dataset.  
- A **plot output** to display the histogram.  
- A **dropdown to select the download format** (PNG or PDF).  
- A **download button** to save the histogram.  

---

### Hints / Guidance

- **Loading the CSV:**  
  Use `fileInput()` and `read.csv()` inside a `reactive()` to read the uploaded file.

- **Dynamic numeric variable selection:**  
  Use `observe()` with `updateSelectInput()` to populate the dropdown after the file is uploaded. Only include numeric columns.

- **Rendering the histogram:**  
  Use `renderPlot()` and `req(input$var)` to ensure a variable is selected before plotting.

- **Downloading the histogram:**  
  Use `downloadHandler()`. Inside `content()`, check the selected format (`png` or `pdf`) and open the corresponding graphics device before plotting.



In [ ]:
library(shiny)

ui <- fluidPage(
  # Upload CSV
  fileInput('upload', 'Upload CSV file', accept = '.csv'),
  
  # Dropdown for numeric variables
  selectInput('var', 'Choose numeric variable', choices = NULL),
  
  # Dropdown for download format
  selectInput('format', 'Choose download format', choices = c('png', 'pdf')),
  
  # Histogram output
  plotOutput('histogram'),
  
  # Download button
  downloadButton('download_plot', 'Download Plot')
)

server <- function(input, output, session) {
  
  # Reactive: read uploaded CSV
  uploaded <- reactive({
    req(input$upload)  # Wait until a file is uploaded
    read.csv(input$upload$datapath)
  })
  
  # Populate numeric variables dropdown after upload
  observe({
    df <- uploaded()
    nums <- names(df)[sapply(df, is.numeric)]
    if(length(nums) == 0) nums <- "No numeric columns"
    updateSelectInput(session, 'var', choices = nums)
  })
  
  # Render histogram
  output$histogram <- renderPlot({
    req(input$var)
    df <- uploaded()
    var <- input$var
    
    # Check column exists and is numeric
    if(!(var %in% names(df)) || !is.numeric(df[[var]])) {
      plot.new()
      text(0.5, 0.5, "Invalid variable selected", cex = 1.5)
      return()
    }
    
    hist(df[[var]], main = paste('Histogram of', var),
         xlab = var, col = 'skyblue', border = 'white')
  })
  
  # Download handler
  output$download_plot <- downloadHandler(
    filename = function() {
      paste0(input$var, ".", input$format)
    },
    content = function(file) {
      df <- uploaded()
      var <- input$var
      
      if(!(var %in% names(df)) || !is.numeric(df[[var]])) return()
      
      if(input$format == "png") {
        png(file, width = 800, height = 600)
      } else {
        pdf(file)
      }
      
      hist(df[[var]], main = paste('Histogram of', var),
           xlab = var, col = 'skyblue', border = 'white')
      dev.off()
    }
  )
}

shinyApp(ui, server)


## 5 — Bookmarking: Saving and Restoring App State

### Learning goals
By the end of this section, you will:
- Understand what **bookmarking** means in Shiny.  
- Know how to make your app **save or share** its current settings using a URL.  
- Be able to restore inputs (like sliders or dropdowns) from a bookmark link.

---

### 5.1 What is bookmarking?

Normally, when you reload a Shiny app, all inputs reset to their defaults.  
**Bookmarking** lets users **save the current app state** — so they can return later or share the same setup with others.

For example:
- You adjust several inputs in your app (e.g., dataset, variable, bin width).  
- You click a **Bookmark** button.  
- A new URL appears that contains the saved state.  
- If you open or share that link, the app reopens with the same settings.

---

### 5.2 How it works

To make bookmarking work, you need to:
1. Turn your `ui` into a **function** — this lets Shiny rebuild the interface when restoring a saved state.  
2. Add a `bookmarkButton()` in the UI.  
3. Enable bookmarking in `shinyApp()` by setting  
   `enableBookmarking = "url"` or `"server"`.  

**URL bookmarking** stores all input values directly in the link (works for simple apps).  
**Server bookmarking** saves the state on the server and gives you a short link (used for large or sensitive data).

---

### 5.3 Example: Bookmark a histogram app

Below is a small app where you can:
- Choose a dataset and numeric variable.  
- Draw a histogram.  
- Save your selection as a bookmark URL to reopen later.


In [ ]:
library(shiny)

# Define UI as a function (required for bookmarking)
ui <- function(request) {
  fluidPage(
    selectInput("dataset", "Choose a dataset",
                choices = c("iris", "mtcars", "faithful")),
    uiOutput("var_ui"),
    plotOutput("hist"),
    bookmarkButton()  # Adds a bookmark button
  )
}

server <- function(input, output, session) {

  # Dynamically create variable selector
  output$var_ui <- renderUI({
    df <- get(input$dataset)
    nums <- names(df)[sapply(df, is.numeric)]
    selectInput("var", "Choose variable", choices = nums)
  })

  # Render histogram
  output$hist <- renderPlot({
    req(input$var)
    df <- get(input$dataset)
    hist(df[[input$var]],
         main = paste("Histogram of", input$var, "from", input$dataset),
         col = "skyblue", border = "white")
  })
}

# Enable bookmarking by URL
shinyApp(ui, server, enableBookmarking = "url")

## 6 — Practical extension: Bookmarking your Histogram App

Now, the previous practical, **extend that app** by adding a **bookmarking feature** — so the user can save their selected settings and return to them later.

---

### 🧩 Your task

Modify your histogram app so that:
1. The app interface is defined as a **function** (`ui <- function(request) {...}`), which is required for bookmarking.  
2. A **Bookmark** button (`bookmarkButton()`) is added to the interface.  
3. Bookmarking is **enabled** using `enableBookmarking = "url"`.  

When users click **Bookmark**, Shiny will generate a URL that encodes the current app state (uploaded file excluded).  
Opening that link will restore all input settings — such as the selected variable and download format.

---

### 💡 Notes
- Uploaded files cannot be bookmarked (for security reasons).  
- However, all **input values** (like dropdown selections) will be saved.  
- This is very useful for apps that use public datasets or fixed data sources.


In [ ]:
# --- UI as a function (required for bookmarking) ---
ui <- function(request) {
  fluidPage(
    titlePanel("Upload, Visualize, and Bookmark"),
    
    fileInput('upload', 'Upload CSV file', accept = '.csv'),
    
    selectInput('var', 'Choose numeric variable', choices = NULL),
    
    selectInput('format', 'Choose download format', choices = c('png', 'pdf')),
    
    plotOutput('histogram'),
    
    downloadButton('download_plot', 'Download Plot'),
    
    bookmarkButton() # Adds the bookmark button
  )
}

# --- Server logic ---
server <- function(input, output, session) {
  
  # Reactive: read uploaded CSV
  uploaded <- reactive({
    req(input$upload)
    read.csv(input$upload$datapath)
  })
  
  # Update dropdown with numeric columns
  observe({
    df <- uploaded()
    nums <- names(df)[sapply(df, is.numeric)]
    if(length(nums) == 0) nums <- "No numeric columns"
    updateSelectInput(session, 'var', choices = nums)
  })
  
  # Render histogram
  output$histogram <- renderPlot({
    req(input$var)
    df <- uploaded()
    var <- input$var
    
    if(!(var %in% names(df)) || !is.numeric(df[[var]])) {
      plot.new()
      text(0.5, 0.5, "Invalid variable selected", cex = 1.5)
      return()
    }
    
    hist(df[[var]], main = paste('Histogram of', var),
         xlab = var, col = 'skyblue', border = 'white')
  })
  
  # Download handler
  output$download_plot <- downloadHandler(
    filename = function() {
      paste0(input$var, ".", input$format)
    },
    content = function(file) {
      df <- uploaded()
      var <- input$var
      
      if(!(var %in% names(df)) || !is.numeric(df[[var]])) return()
      
      if(input$format == "png") {
        png(file, width = 800, height = 600)
      } else {
        pdf(file)
      }
      
      hist(df[[var]], main = paste('Histogram of', var),
           xlab = var, col = 'skyblue', border = 'white')
      dev.off()
    }
  )
}

# --- Enable bookmarking by URL ---
shinyApp(ui, server, enableBookmarking = "url")

## 7 — Deploying your Shiny app online with Shinyapps.io

So far, you’ve been running your Shiny apps **locally** — they open in a web browser, but only on your own computer.  
Now you’ll learn how to **publish** your app to the web so anyone can access it from a link, using **[Shinyapps.io](https://www.shinyapps.io/)**.

---

### 🌐 What is Shinyapps.io?

**Shinyapps.io** is a free hosting platform provided by **Posit (formerly RStudio)** that allows you to:
- Deploy your Shiny apps directly from R.
- Host them on the web without needing your own server.
- Share them via a simple web URL (e.g., `https://yourname.shinyapps.io/myapp`).

---

### 🧰 What you need
Before deployment, make sure you have:
- An account on [https://www.shinyapps.io](https://www.shinyapps.io)  
- The **`rsconnect`** R package installed  


In [ ]:
# Install it if needed:
install.packages("rsconnect")

✅ Example

If your previous histogram app is saved as `app.R`, just run this:

In [ ]:
install.packages("rsconnect")  # if not already installed
library(rsconnect)

rsconnect::setAccountInfo(name='yourname',
                          token='ABCDEF123456',
                          secret='XYZ987654321')

rsconnect::deployApp('path/to/your/histogram_app')
